In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
df_ml = pd.read_csv('../data/cleaned_data/student_social_media_addiction_cleaned.csv')

In [10]:
df_ml.columns

Index(['new_student_id', 'age', 'gender', 'relationship_status',
       'academic_level', 'country', 'continents', 'most_used_platform',
       'average_daily_usage_minutes', 'average_daily_use_by_hour',
       'sleep_per_night_minutes', 'sleep_hours_per_night',
       'conflicts_over_social_media', 'affects_academic_performance',
       'addicted_score', 'mental_health_score'],
      dtype='object')

In [11]:
num_cols = ['age', 'average_daily_usage_minutes', 'sleep_per_night_minutes', 'conflicts_over_social_media', 'addicted_score', 'mental_health_score']
cat_cols = ['gender', 'relationship_status', 'academic_level', 'country', 'continents', 'most_used_platform', 'affects_academic_performance']

In [14]:
preprocessor = ColumnTransformer([
    ('numeric', StandardScaler(), num_cols),
    ('categorical', OneHotEncoder(drop='first', sparse=False), cat_cols)
])